Installing required libraries for the project
yfinance for the stock market data
seaborn for visualization
sklearn for ML

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import json as json
import joblib
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
import matplotlib.pyplot as plt



Impport data for this project:
- TSLA Stock Price
- BTC Price
- DOGE Price
- News before Market open time
- Top News before Market open time

In [ ]:
#Balance Sheet data is not mandatory and the files are already HUGE
Stock = pd.read_csv('Data\\us-shareprices-daily.csv', sep=";")
#Balance = pd.read_csv('Data\\us-shareprices-daily.csv', sep=";")
Companies = pd.read_csv('Data\\us-balance-quarterly.csv', sep=";")


In [ ]:
Stock.columns

Index(['Ticker', 'SimFinId', 'Date', 'Open', 'High', 'Low', 'Close',
       'Adj. Close', 'Volume', 'Dividend', 'Shares Outstanding'],
      dtype='object')

In [ ]:
Companies.columns

Index(['Ticker', 'SimFinId', 'Currency', 'Fiscal Year', 'Fiscal Period',
       'Report Date', 'Publish Date', 'Restated Date', 'Shares (Basic)',
       'Shares (Diluted)', 'Cash, Cash Equivalents & Short Term Investments',
       'Accounts & Notes Receivable', 'Inventories', 'Total Current Assets',
       'Property, Plant & Equipment, Net',
       'Long Term Investments & Receivables', 'Other Long Term Assets',
       'Total Noncurrent Assets', 'Total Assets', 'Payables & Accruals',
       'Short Term Debt', 'Total Current Liabilities', 'Long Term Debt',
       'Total Noncurrent Liabilities', 'Total Liabilities',
       'Share Capital & Additional Paid-In Capital', 'Treasury Stock',
       'Retained Earnings', 'Total Equity', 'Total Liabilities & Equity'],
      dtype='object')

In [ ]:
Stock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5767359 entries, 0 to 5767358
Data columns (total 11 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Ticker              object 
 1   SimFinId            int64  
 2   Date                object 
 3   Open                float64
 4   High                float64
 5   Low                 float64
 6   Close               float64
 7   Adj. Close          float64
 8   Volume              int64  
 9   Dividend            float64
 10  Shares Outstanding  float64
dtypes: float64(7), int64(2), object(2)
memory usage: 484.0+ MB


In [ ]:
Stock['Date']= pd.to_datetime(Stock['Date'])

In [ ]:
Stock['Fiscal Year'] = Stock['Date'].dt.year
Stock['Fiscal Period'] = Stock['Date'].dt.to_period('Q').dt.strftime('Q%q')


In [ ]:
Stock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5767359 entries, 0 to 5767358
Data columns (total 13 columns):
 #   Column              Dtype         
---  ------              -----         
 0   Ticker              object        
 1   SimFinId            int64         
 2   Date                datetime64[ns]
 3   Open                float64       
 4   High                float64       
 5   Low                 float64       
 6   Close               float64       
 7   Adj. Close          float64       
 8   Volume              int64         
 9   Dividend            float64       
 10  Shares Outstanding  float64       
 11  Fiscal Year         int32         
 12  Fiscal Period       object        
dtypes: datetime64[ns](1), float64(7), int32(1), int64(2), object(2)
memory usage: 550.0+ MB


In [ ]:
Companies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55992 entries, 0 to 55991
Data columns (total 30 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   Ticker                                           55992 non-null  object 
 1   SimFinId                                         55992 non-null  int64  
 2   Currency                                         55992 non-null  object 
 3   Fiscal Year                                      55992 non-null  int64  
 4   Fiscal Period                                    55992 non-null  object 
 5   Report Date                                      55992 non-null  object 
 6   Publish Date                                     55992 non-null  object 
 7   Restated Date                                    55992 non-null  object 
 8   Shares (Basic)                                   55763 non-null  float64
 9   Shares (Diluted)            

In [ ]:
Stock = Stock[Stock['Ticker'].isin(['TSLA', 'AAPL', 'MSFT', 'NVDA', 'META'])]
Companies = Companies[Companies['Ticker'].isin(['TSLA', 'AAPL', 'MSFT', 'NVDA', 'META'])]


*Next Steps:*
- Combine all together then filter the data to keep only the selected (5) tickers
- Clean the data
- Delete unuseful columns
- Make sure that all data types are correct


In [ ]:
combined = pd.merge(Stock,Companies, on=['Ticker','Fiscal Year','Fiscal Period'], how='left')
combined

,Ticker,SimFinId_x,Date,Open,High,Low,Close,Adj. Close,Volume,Dividend,...,Short Term Debt,Total Current Liabilities,Long Term Debt,Total Noncurrent Liabilities,Total Liabilities,Share Capital & Additional Paid-In Capital,Treasury Stock,Retained Earnings,Total Equity,Total Liabilities & Equity
0,AAPL,111052,2019-04-01,47.91,47.92,47.09,47.81,45.81,111447856,NaN,...,2.244798e+10,9.385251e+10,9.010885e+10,5.216500e+10,2.361380e+11,4.280100e+10,NaN,6.455800e+10,1.058600e+11,3.419980e+11
1,AAPL,111052,2019-04-02,47.77,48.62,47.76,48.51,46.47,91062928,NaN,...,2.244798e+10,9.385251e+10,9.010885e+10,5.216500e+10,2.361380e+11,4.280100e+10,NaN,6.455800e+10,1.058600e+11,3.419980e+11
2,AAPL,111052,2019-04-03,48.31,49.12,48.29,48.84,46.79,93087320,NaN,...,2.244798e+10,9.385251e+10,9.010885e+10,5.216500e+10,2.361380e+11,4.280100e+10,NaN,6.455800e+10,1.058600e+11,3.419980e+11
3,AAPL,111052,2019-04-04,48.70,49.09,48.28,48.92,46.87,76457100,NaN,...,2.244798e+10,9.385251e+10,9.010885e+10,5.216500e+10,2.361380e+11,4.280100e+10,NaN,6.455800e+10,1.058600e+11,3.419980e+11
4,AAPL,111052,2019-04-05,49.11,49.27,48.98,49.25,47.19,74106576,NaN,...,2.244798e+10,9.385251e+10,9.010885e+10,5.216500e+10,2.361380e+11,4.280100e+10,NaN,6.455800e+10,1.058600e+11,3.419980e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6190,TSLA,56317,2024-02-26,192.29,201.78,192.00,199.40,199.40,111747116,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6191,TSLA,56317,2024-02-27,204.04,205.60,198.26,199.73,199.73,108645412,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6192,TSLA,56317,2024-02-28,200.42,205.30,198.44,202.04,202.04,99806173,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6193,TSLA,56317,2024-02-29,204.18,205.28,198.45,201.88,201.88,85906974,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6195 entries, 0 to 6194
Data columns (total 40 columns):
 #   Column                                           Non-Null Count  Dtype         
---  ------                                           --------------  -----         
 0   Ticker                                           6195 non-null   object        
 1   SimFinId_x                                       6195 non-null   int64         
 2   Date                                             6195 non-null   datetime64[ns]
 3   Open                                             6195 non-null   float64       
 4   High                                             6195 non-null   float64       
 5   Low                                              6195 non-null   float64       
 6   Close                                            6195 non-null   float64       
 7   Adj. Close                                       6195 non-null   float64       
 8   Volume                                

In [ ]:
combined.isnull().sum()

Ticker                                                0
SimFinId_x                                            0
Date                                                  0
Open                                                  0
High                                                  0
Low                                                   0
Close                                                 0
Adj. Close                                            0
Volume                                                0
Dividend                                           6135
Shares Outstanding                                    0
Fiscal Year                                           0
Fiscal Period                                         0
SimFinId_y                                          189
Currency                                            189
Report Date                                         189
Publish Date                                        189
Restated Date                                   

In [ ]:
combined.duplicated().sum()

0

In [ ]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6195 entries, 0 to 6194
Data columns (total 40 columns):
 #   Column                                           Non-Null Count  Dtype         
---  ------                                           --------------  -----         
 0   Ticker                                           6195 non-null   object        
 1   SimFinId_x                                       6195 non-null   int64         
 2   Date                                             6195 non-null   datetime64[ns]
 3   Open                                             6195 non-null   float64       
 4   High                                             6195 non-null   float64       
 5   Low                                              6195 non-null   float64       
 6   Close                                            6195 non-null   float64       
 7   Adj. Close                                       6195 non-null   float64       
 8   Volume                                

In [ ]:
combined.select_dtypes(include=['object'])

,Ticker,Fiscal Period,Currency,Report Date,Publish Date,Restated Date
0,AAPL,Q2,USD,2019-03-31,2019-05-01,2019-05-01
1,AAPL,Q2,USD,2019-03-31,2019-05-01,2019-05-01
2,AAPL,Q2,USD,2019-03-31,2019-05-01,2019-05-01
3,AAPL,Q2,USD,2019-03-31,2019-05-01,2019-05-01
4,AAPL,Q2,USD,2019-03-31,2019-05-01,2019-05-01
...,...,...,...,...,...,...
6190,TSLA,Q1,NaN,NaN,NaN,NaN
6191,TSLA,Q1,NaN,NaN,NaN,NaN
6192,TSLA,Q1,NaN,NaN,NaN,NaN
6193,TSLA,Q1,NaN,NaN,NaN,NaN


In [ ]:
combined.columns

Index(['Ticker', 'SimFinId_x', 'Date', 'Open', 'High', 'Low', 'Close',
       'Adj. Close', 'Volume', 'Dividend', 'Shares Outstanding', 'Fiscal Year',
       'Fiscal Period', 'SimFinId_y', 'Currency', 'Report Date',
       'Publish Date', 'Restated Date', 'Shares (Basic)', 'Shares (Diluted)',
       'Cash, Cash Equivalents & Short Term Investments',
       'Accounts & Notes Receivable', 'Inventories', 'Total Current Assets',
       'Property, Plant & Equipment, Net',
       'Long Term Investments & Receivables', 'Other Long Term Assets',
       'Total Noncurrent Assets', 'Total Assets', 'Payables & Accruals',
       'Short Term Debt', 'Total Current Liabilities', 'Long Term Debt',
       'Total Noncurrent Liabilities', 'Total Liabilities',
       'Share Capital & Additional Paid-In Capital', 'Treasury Stock',
       'Retained Earnings', 'Total Equity', 'Total Liabilities & Equity'],
      dtype='object')

In [ ]:
combined['Date'] = pd.to_datetime(combined['Date'])
combined['Report Date'] = pd.to_datetime(combined['Report Date'])
combined['Publish Date'] = pd.to_datetime(combined['Publish Date'])
combined['Restated Date'] = pd.to_datetime(combined['Restated Date'])
combined.select_dtypes(include=['object'])

,Ticker,Fiscal Period,Currency
0,AAPL,Q2,USD
1,AAPL,Q2,USD
2,AAPL,Q2,USD
3,AAPL,Q2,USD
4,AAPL,Q2,USD
...,...,...,...
6190,TSLA,Q1,NaN
6191,TSLA,Q1,NaN
6192,TSLA,Q1,NaN
6193,TSLA,Q1,NaN


In [ ]:
combined['Ticker'].unique()

array(['AAPL', 'META', 'MSFT', 'NVDA', 'TSLA'], dtype=object)

In [ ]:
combined['week'] = combined['Date'].dt.isocalendar().week
combined['First_day'] = combined['week']!=combined['week'].shift(1)
combined['Last_day'] = combined['week']!= combined['week'].shift(-1)

In [ ]:
combined['First_day'].fillna(False, inplace=True)
combined['Last_day'].fillna(False, inplace=True)

C:\Users\omara\AppData\Local\Temp\ipykernel_3436\3479062945.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  combined['First_day'].fillna(False, inplace=True)
C:\Users\omara\AppData\Local\Temp\ipykernel_3436\3479062945.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exam

In [ ]:
import pandas as pd

# Assuming 'combined' is your DataFrame and it has columns named 'Ticker', 'Close', and 'Volume'
# combined = pd.read_csv("your_data.csv")

def calculate_rsi(d, window):
    delta = d['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calculate_indicators(group):
    group['SMA_7'] = group['Close'].rolling(window=7).mean()
    group['V_SMA_7'] = group['Volume'].rolling(window=7).mean()
    group['SMA_14'] = group['Close'].rolling(window=14).mean()
    group['V_SMA_14'] = group['Volume'].rolling(window=14).mean()
    group['EMA_7'] = group['Close'].ewm(span=7, adjust=False).mean()
    group['V_EMA_7'] = group['Volume'].ewm(span=7, adjust=False).mean()
    group['EMA_14'] = group['Close'].ewm(span=14, adjust=False).mean()
    group['V_EMA_14'] = group['Volume'].ewm(span=14, adjust=False).mean()

    # Calculate RSI for 7 and 14 days and add it to the group
    #group['RSI_7'] = calculate_rsi(group, 7)
    #group['RSI_14'] = calculate_rsi(group, 14)

    return group

# Group by 'Ticker' and apply the function
combined_grouped = combined.groupby('Ticker')
combined = combined_grouped.apply(calculate_indicators)




C:\Users\omara\AppData\Local\Temp\ipykernel_3436\1072301618.py:32: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  combined = combined_grouped.apply(calculate_indicators)


In [ ]:
combined.isnull().sum()

Ticker                                                0
SimFinId_x                                            0
Date                                                  0
Open                                                  0
High                                                  0
Low                                                   0
Close                                                 0
Adj. Close                                            0
Volume                                                0
Dividend                                           6135
Shares Outstanding                                    0
Fiscal Year                                           0
Fiscal Period                                         0
SimFinId_y                                          189
Currency                                            189
Report Date                                         189
Publish Date                                        189
Restated Date                                   

In [ ]:
TSLA = combined[combined['Ticker']=='TSLA']

In [ ]:
TSLA.columns

Index(['Ticker', 'SimFinId_x', 'Date', 'Open', 'High', 'Low', 'Close',
       'Adj. Close', 'Volume', 'Dividend', 'Shares Outstanding', 'Fiscal Year',
       'Fiscal Period', 'SimFinId_y', 'Currency', 'Report Date',
       'Publish Date', 'Restated Date', 'Shares (Basic)', 'Shares (Diluted)',
       'Cash, Cash Equivalents & Short Term Investments',
       'Accounts & Notes Receivable', 'Inventories', 'Total Current Assets',
       'Property, Plant & Equipment, Net',
       'Long Term Investments & Receivables', 'Other Long Term Assets',
       'Total Noncurrent Assets', 'Total Assets', 'Payables & Accruals',
       'Short Term Debt', 'Total Current Liabilities', 'Long Term Debt',
       'Total Noncurrent Liabilities', 'Total Liabilities',
       'Share Capital & Additional Paid-In Capital', 'Treasury Stock',
       'Retained Earnings', 'Total Equity', 'Total Liabilities & Equity',
       'week', 'First_day', 'Last_day', 'SMA_7', 'V_SMA_7', 'SMA_14',
       'V_SMA_14', 'EMA_7', 'V_EMA_

In this step, we are looking at the outcome of the caluclated fields making sure that it's working properly.

In [ ]:
TSLA[['Date','SMA_7', 'V_SMA_7', 'SMA_14', 'V_SMA_14', 'EMA_7',
       'V_EMA_7', 'EMA_14', 'V_EMA_14']]

Date       SMA_7       V_SMA_7      SMA_14      V_SMA_14  \
Ticker                                                                       
TSLA   4956 2019-04-01         NaN           NaN         NaN           NaN   
       4957 2019-04-02         NaN           NaN         NaN           NaN   
       4958 2019-04-03         NaN           NaN         NaN           NaN   
       4959 2019-04-04         NaN           NaN         NaN           NaN   
       4960 2019-04-05         NaN           NaN         NaN           NaN   
...                ...         ...           ...         ...           ...   
       6190 2024-02-26  196.815714  1.034138e+08  192.455714  9.921995e+07   
       6191 2024-02-27  196.712857  1.016729e+08  193.500714  9.821777e+07   
       6192 2024-02-28  197.011429  1.000243e+08  194.533571  9.737998e+07   
       6193 2024-02-29  198.171429  9.736158e+07  195.413571  9.758519e+07   
       6194 2024-03-01  199.295714  9.427574e+07  196.061429  9.742568e+07   

                  EMA_7       V_EMA_7      EMA_14      V_EMA_14  
Ticker                                                           
TSLA   4956   19.280000  1.216566e+08   19.280000  1.216566e+08  
       4957   19.225000  1.117885e+08   19.250667  1.163936e+08  
       4958   19.281250  1.135783e+08   19.277244  1.167342e+08  
       4959   18.923437  1.741365e+08   19.086945  1.486111e+08  
       4960   18.775078  1.794958e+08   18.986019  1.548728e+08  
...                 ...           ...         ...           ...  
       6190  195.366817  9.939424e+07  194.724306  1.012069e+08  
       6191  196.457613  1.017070e+08  195.391732  1.021987e+08  
       6192  197.853210  1.012318e+08  196.278168  1.018797e+08  
       6193  198.859907  9.740061e+07  197.025079  9.974997e+07  
       6194  199.804930  9.361123e+07  197.773735  9.741573e+07  

[1239 rows x 9 columns]

Now by this we have completed the ETL process. However, the null values were not yet handled, and the reason behind this we want to look at it all together part of the ML proecss and from there decide how to handle them.

In [ ]:
combined.to_csv('Combined_data')